In [1]:

%pylab inline
import catboost
import pandas as pd
import pickle
import tqdm

import seaborn as sns


def percentile_borders(series):
    return [int(series.quantile(perc)) for perc in [0.05, 0.2, 0.3, 0.4, 0.5, 0.65, 0.8, 0.95]]

Populating the interactive namespace from numpy and matplotlib


In [2]:
set_name = 'set2'
path_train_set = 'taxi-idao/data/train/{}.csv'.format(set_name)

data = pd.read_csv(path_train_set)
data.datetime = data.datetime.apply(
    lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
data = data.sort_values('datetime')
data.head()

,datetime,num_orders
0,2018-04-01 00:00:00,0
1,2018-04-01 00:01:00,0
2,2018-04-01 00:02:00,0
3,2018-04-01 00:03:00,0
4,2018-04-01 00:04:00,0


In [3]:
target_positions = {
    'set1': [10, 30, 45, 60, 75],
    'set2': [5, 10, 15, 20, 25],
    'set3': [5, 7, 9, 11, 13]
}[set_name]



HOUR_IN_MINUTES = 60
DAY_IN_MINUTES = 24 * HOUR_IN_MINUTES
WEEK_IN_MINUTES = 7 * DAY_IN_MINUTES

MAX_TIME = DAY_IN_MINUTES


samples = {
    'datetime': [],
    'history': []}

for position in target_positions:
    samples['target_{}'.format(position)] = []
    
num_orders = data.num_orders.values



# start after 2 weeks because of history
# finish earlier because of target calculation
for i in range(2 * WEEK_IN_MINUTES,
               len(num_orders) - 2 * DAY_IN_MINUTES):
    
    samples['datetime'].append(data.datetime[i])
    samples['history'].append(num_orders[i-2*WEEK_IN_MINUTES:i])
    
    # cumsum not for all array because of time economy
    cumsum_num_orders = num_orders[i+1:i+1+2*DAY_IN_MINUTES].cumsum()
    for position in target_positions:
        orders_by_positions = np.where(cumsum_num_orders >= position)[0]
        if len(orders_by_positions):
            time = orders_by_positions[0] + 1
        else:
            # if no orders in last days
            time = MAX_TIME
        samples['target_{}'.format(position)].append(time)

In [4]:
df = pd.DataFrame.from_dict(samples)
df.head()

,datetime,history,target_5,target_10,target_15,target_20,target_25
0,2018-04-15 00:00:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",205,221,247,285,297
1,2018-04-15 00:01:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",204,220,246,284,296
2,2018-04-15 00:02:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",203,219,245,283,295
3,2018-04-15 00:03:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",202,218,244,282,294
4,2018-04-15 00:04:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",201,217,243,281,293


## Base features

In [5]:
df['weekday'] = df.datetime.apply(lambda x: x.weekday())
df['hour'] = df.datetime.apply(lambda x: x.hour)
df['minute'] = df.datetime.apply(lambda x: x.minute)


SHIFTS = [
    HOUR_IN_MINUTES // 4,
    HOUR_IN_MINUTES // 2,
    HOUR_IN_MINUTES,
    DAY_IN_MINUTES,
    DAY_IN_MINUTES * 2,
    WEEK_IN_MINUTES,
    WEEK_IN_MINUTES * 2]
WINDOWS = [
    HOUR_IN_MINUTES // 4 - 1,
    HOUR_IN_MINUTES // 2 - 1,
    HOUR_IN_MINUTES - 1,
    DAY_IN_MINUTES - 1,
    DAY_IN_MINUTES * 2 - 1,
    WEEK_IN_MINUTES - 1,
    WEEK_IN_MINUTES * 2 - 1]


all_features_names = ['weekday', 'hour', 'minute']
for shift in SHIFTS:
    for window in WINDOWS:
        if window > shift:
            continue
        all_features_names.append('num_orders_{}_{}'.format(shift, window))
        df['num_orders_{}_{}'.format(shift, window)] = \
            df.history.apply(lambda x: x[-shift : -shift + window].sum())

## Percentiles

In [6]:
windows = {}
for tg in target_positions:
    windows[tg] = percentile_borders(df['target_' + str(tg)])
windows_names = ['5%', '20%', '30%', '40%', '50%', '65%', '80%', '95%']
windows

{5: [10, 23, 34, 48, 65, 101, 160, 282],
 10: [22, 49, 69, 93, 119, 173, 250, 386],
 15: [35, 75, 105, 136, 171, 235, 319, 457],
 20: [51, 104, 141, 181, 223, 291, 378, 515],
 25: [67, 133, 177, 223, 271, 346, 431, 568]}

In [7]:
SHIFTS_PERCENTILE = [WEEK_IN_MINUTES, WEEK_IN_MINUTES * 2, DAY_IN_MINUTES]
WINDOWS_PERCENTILE = {5: [10, 23, 34, 48, 65, 101, 160, 282],
                      10: [22, 49, 69, 93, 119, 173, 250, 386],
                      15: [35, 75, 105, 136, 171, 235, 319, 457],
                      20: [51, 104, 141, 181, 223, 291, 378, 515],
                      25: [67, 133, 177, 223, 271, 346, 431, 568]}



shifts_names = ['week_ago', 'two_weeks_ago', 'day_ago']
windows_names = ['5%', '20%', '30%', '40%', '50%', '65%', '80%', '95%']

In [8]:
### part 1

In [9]:
df_history = df.history.values

for target in tqdm.tqdm(target_positions):
    for i, shift in enumerate(SHIFTS_PERCENTILE):
        for j, window in enumerate(WINDOWS_PERCENTILE[target]):
            df['num_orders_target_{}_{}_{}'.format(target, shifts_names[i], windows_names[j])] = list(map(lambda x: x[-shift: -shift + window].sum(), df_history))

100%|██████████| 5/5 [02:05<00:00, 25.03s/it]


In [10]:
### part 2

In [11]:
for target in tqdm.tqdm(target_positions):
    for j, window in enumerate(WINDOWS_PERCENTILE[target]):
        col_name = 'num_orders_target_{}_previous_{}_perc'.format(target, windows_names[j])
        df[col_name] = list(map(lambda x: x[-window:].sum(), df_history))

100%|██████████| 5/5 [00:41<00:00,  8.34s/it]


In [12]:
df.head(2)

,datetime,history,target_5,target_10,target_15,target_20,target_25,weekday,hour,minute,...,num_orders_target_20_previous_80%_perc,num_orders_target_20_previous_95%_perc,num_orders_target_25_previous_5%_perc,num_orders_target_25_previous_20%_perc,num_orders_target_25_previous_30%_perc,num_orders_target_25_previous_40%_perc,num_orders_target_25_previous_50%_perc,num_orders_target_25_previous_65%_perc,num_orders_target_25_previous_80%_perc,num_orders_target_25_previous_95%_perc
0,2018-04-15 00:00:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",205,221,247,285,297,6,0,0,...,7,21,0,1,3,3,3,5,14,21
1,2018-04-15 00:01:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",204,220,246,284,296,6,0,1,...,6,21,0,1,3,3,3,5,14,21


# Create list

In [18]:
ALL_FEATURES = df.columns[7:]

In [19]:
len(ALL_FEATURES)

191

In [20]:
ALL_FEATURES[:31]

Index(['weekday', 'hour', 'minute', 'num_orders_15_14', 'num_orders_30_14',
       'num_orders_30_29', 'num_orders_60_14', 'num_orders_60_29',
       'num_orders_60_59', 'num_orders_1440_14', 'num_orders_1440_29',
       'num_orders_1440_59', 'num_orders_1440_1439', 'num_orders_2880_14',
       'num_orders_2880_29', 'num_orders_2880_59', 'num_orders_2880_1439',
       'num_orders_2880_2879', 'num_orders_10080_14', 'num_orders_10080_29',
       'num_orders_10080_59', 'num_orders_10080_1439', 'num_orders_10080_2879',
       'num_orders_10080_10079', 'num_orders_20160_14', 'num_orders_20160_29',
       'num_orders_20160_59', 'num_orders_20160_1439', 'num_orders_20160_2879',
       'num_orders_20160_10079', 'num_orders_20160_20159'],
      dtype='object')

In [23]:
TARGET_FEATURES_POSITIONS = {}
TARGET_FEATURES_POSITIONS[5] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[10] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[15] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[20] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[25] = [i for i in range(31)]


TARGET_FEATURES_POSITIONS[5] = TARGET_FEATURES_POSITIONS[5] + [i for i in range(31,31+24)]
TARGET_FEATURES_POSITIONS[10] = TARGET_FEATURES_POSITIONS[10] + [i for i in range(31+24,31+24*2)]
TARGET_FEATURES_POSITIONS[15] = TARGET_FEATURES_POSITIONS[15] + [i for i in range(31+24*2,31+24*3)]
TARGET_FEATURES_POSITIONS[20] = TARGET_FEATURES_POSITIONS[20] + [i for i in range(31+24*3,31+24*4)]
TARGET_FEATURES_POSITIONS[25] = TARGET_FEATURES_POSITIONS[25] + [i for i in range(31+24*4,31+24*5)]


TARGET_FEATURES_POSITIONS[5] = TARGET_FEATURES_POSITIONS[5] + [i for i in range(151,151+8)] + [191]
TARGET_FEATURES_POSITIONS[10] = TARGET_FEATURES_POSITIONS[10] + [i for i in range(151+8,151+8*2)] + [191]
TARGET_FEATURES_POSITIONS[15] = TARGET_FEATURES_POSITIONS[15] + [i for i in range(151+8*2,151+8*3)] + [191]
TARGET_FEATURES_POSITIONS[20] = TARGET_FEATURES_POSITIONS[20] + [i for i in range(151+8*3,151+8*4)] + [191]
TARGET_FEATURES_POSITIONS[25] = TARGET_FEATURES_POSITIONS[25] + [i for i in range(151+8*4,151+8*5)] + [191]

In [25]:
ALL_FEATURES[TARGET_FEATURES_POSITIONS[25]]

Index(['weekday', 'hour', 'minute', 'num_orders_15_14', 'num_orders_30_14',
       'num_orders_30_29', 'num_orders_60_14', 'num_orders_60_29',
       'num_orders_60_59', 'num_orders_1440_14', 'num_orders_1440_29',
       'num_orders_1440_59', 'num_orders_1440_1439', 'num_orders_2880_14',
       'num_orders_2880_29', 'num_orders_2880_59', 'num_orders_2880_1439',
       'num_orders_2880_2879', 'num_orders_10080_14', 'num_orders_10080_29',
       'num_orders_10080_59', 'num_orders_10080_1439', 'num_orders_10080_2879',
       'num_orders_10080_10079', 'num_orders_20160_14', 'num_orders_20160_29',
       'num_orders_20160_59', 'num_orders_20160_1439', 'num_orders_20160_2879',
       'num_orders_20160_10079', 'num_orders_20160_20159',
       'num_orders_target_25_week_ago_5%', 'num_orders_target_25_week_ago_20%',
       'num_orders_target_25_week_ago_30%',
       'num_orders_target_25_week_ago_40%',
       'num_orders_target_25_week_ago_50%',
       'num_orders_target_25_week_ago_65%',
     

# Split data

In [27]:
df.datetime.min(), df.datetime.max()
df_train = df.loc[df.datetime <= df.datetime.max() - datetime.timedelta(days=28)]
df_test = df.loc[(df.datetime > df.datetime.max() - datetime.timedelta(days=14))]



target_cols = ['target_{}'.format(position) for position in target_positions]

y_train = df_train[target_cols]
y_test = df_test[target_cols]

df_train = df_train.drop(['datetime', 'history'] + target_cols, axis=1)
df_test = df_test.drop(['datetime', 'history'] + target_cols, axis=1)



def sMAPE(y_true, y_predict, shift=0):
    return 2 * np.mean(
        np.abs(y_true - y_predict) /
        (np.abs(y_true) + np.abs(y_predict) + shift))


model_to_save = {
    'models': {}
}

# Train for validation

In [34]:
### all features
for position in target_positions:
    model = catboost.CatBoostRegressor(
        iterations=2000, learning_rate=1.0, loss_function='MAE')
    model.fit(
        X=df_train[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]],
        y=y_train['target_{}'.format(position)],
        use_best_model=True,
        eval_set=(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]], y_test['target_{}'.format(position)]),
        verbose=True)
    y_predict = model.predict(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]])
    
    print('target_{}'.format(position))
    print('stupid:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_train['target_{}'.format(position)].median())))
    print('model:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_predict)))
    print()
    
    model_to_save['models'][position] = model

0:	learn: 99.0951189	test: 71.6081003	best: 71.6081003 (0)	total: 66.3ms	remaining: 2m 12s
1:	learn: 98.5951339	test: 71.1081153	best: 71.1081153 (1)	total: 123ms	remaining: 2m 3s
2:	learn: 98.0951888	test: 70.6082295	best: 70.6082295 (2)	total: 203ms	remaining: 2m 15s
3:	learn: 97.5953711	test: 70.1085042	best: 70.1085042 (3)	total: 372ms	remaining: 3m 5s
4:	learn: 97.0960632	test: 69.6097276	best: 69.6097276 (4)	total: 571ms	remaining: 3m 48s
5:	learn: 96.5971654	test: 69.1113297	best: 69.1113297 (5)	total: 733ms	remaining: 4m 3s
6:	learn: 96.0999229	test: 68.6154764	best: 68.6154764 (6)	total: 888ms	remaining: 4m 12s
7:	learn: 95.6043731	test: 68.1216090	best: 68.1216090 (7)	total: 1.05s	remaining: 4m 21s
8:	learn: 95.1121593	test: 67.6322249	best: 67.6322249 (8)	total: 1.19s	remaining: 4m 24s
9:	learn: 94.6235314	test: 67.1467021	best: 67.1467021 (9)	total: 1.35s	remaining: 4m 29s
10:	learn: 94.1390496	test: 66.6663247	best: 66.6663247 (10)	total: 1.61s	remaining: 4m 51s
11:	learn:

90:	learn: 69.6770413	test: 45.2811041	best: 45.2811041 (90)	total: 18.5s	remaining: 6m 28s
91:	learn: 69.4839166	test: 45.1450316	best: 45.1450316 (91)	total: 18.7s	remaining: 6m 28s
92:	learn: 69.2924852	test: 45.0097427	best: 45.0097427 (92)	total: 18.9s	remaining: 6m 27s
93:	learn: 69.1028788	test: 44.8772892	best: 44.8772892 (93)	total: 19.1s	remaining: 6m 26s
94:	learn: 68.9151136	test: 44.7401663	best: 44.7401663 (94)	total: 19.3s	remaining: 6m 26s
95:	learn: 68.7289057	test: 44.6088300	best: 44.6088300 (95)	total: 19.5s	remaining: 6m 26s
96:	learn: 68.5444220	test: 44.4778264	best: 44.4778264 (96)	total: 19.7s	remaining: 6m 25s
97:	learn: 68.3615402	test: 44.3520655	best: 44.3520655 (97)	total: 19.9s	remaining: 6m 25s
98:	learn: 68.1803408	test: 44.2254424	best: 44.2254424 (98)	total: 20s	remaining: 6m 24s
99:	learn: 68.0007018	test: 44.0992015	best: 44.0992015 (99)	total: 20.3s	remaining: 6m 26s
100:	learn: 67.8221377	test: 43.9754351	best: 43.9754351 (100)	total: 20.6s	remain

179:	learn: 57.2880386	test: 37.6876516	best: 37.6876516 (179)	total: 32.5s	remaining: 5m 28s
180:	learn: 57.1884111	test: 37.6442758	best: 37.6442758 (180)	total: 32.7s	remaining: 5m 28s
181:	learn: 57.0896136	test: 37.6010286	best: 37.6010286 (181)	total: 32.8s	remaining: 5m 28s
182:	learn: 56.9918652	test: 37.5540644	best: 37.5540644 (182)	total: 33s	remaining: 5m 27s
183:	learn: 56.8951157	test: 37.5038611	best: 37.5038611 (183)	total: 33.2s	remaining: 5m 27s
184:	learn: 56.7973112	test: 37.4589615	best: 37.4589615 (184)	total: 33.4s	remaining: 5m 27s
185:	learn: 56.7002996	test: 37.4207905	best: 37.4207905 (185)	total: 33.5s	remaining: 5m 27s
186:	learn: 56.6037702	test: 37.3819736	best: 37.3819736 (186)	total: 33.7s	remaining: 5m 26s
187:	learn: 56.5083662	test: 37.3436619	best: 37.3436619 (187)	total: 33.9s	remaining: 5m 26s
188:	learn: 56.4141100	test: 37.3008224	best: 37.3008224 (188)	total: 34s	remaining: 5m 26s
189:	learn: 56.3198635	test: 37.2637137	best: 37.2637137 (189)	t

267:	learn: 50.4445455	test: 35.2818706	best: 35.2818706 (267)	total: 48.2s	remaining: 5m 11s
268:	learn: 50.3840668	test: 35.2673285	best: 35.2673285 (268)	total: 48.4s	remaining: 5m 11s
269:	learn: 50.3242238	test: 35.2539031	best: 35.2539031 (269)	total: 48.5s	remaining: 5m 10s
270:	learn: 50.2653225	test: 35.2411490	best: 35.2411490 (270)	total: 48.7s	remaining: 5m 10s
271:	learn: 50.2070090	test: 35.2285974	best: 35.2285974 (271)	total: 48.9s	remaining: 5m 10s
272:	learn: 50.1479960	test: 35.2147529	best: 35.2147529 (272)	total: 49s	remaining: 5m 10s
273:	learn: 50.0892848	test: 35.2005682	best: 35.2005682 (273)	total: 49.2s	remaining: 5m 9s
274:	learn: 50.0323697	test: 35.1898281	best: 35.1898281 (274)	total: 49.4s	remaining: 5m 9s
275:	learn: 49.9752940	test: 35.1781712	best: 35.1781712 (275)	total: 49.5s	remaining: 5m 9s
276:	learn: 49.9194674	test: 35.1607882	best: 35.1607882 (276)	total: 49.7s	remaining: 5m 9s
277:	learn: 49.8632335	test: 35.1477551	best: 35.1477551 (277)	tot

357:	learn: 46.0583971	test: 34.5514569	best: 34.5514569 (357)	total: 1m 5s	remaining: 5m 1s
358:	learn: 46.0178583	test: 34.5497430	best: 34.5497430 (358)	total: 1m 5s	remaining: 5m
359:	learn: 45.9764017	test: 34.5395374	best: 34.5395374 (359)	total: 1m 6s	remaining: 5m 1s
360:	learn: 45.9374123	test: 34.5343684	best: 34.5343684 (360)	total: 1m 6s	remaining: 5m 1s
361:	learn: 45.8974641	test: 34.5349970	best: 34.5343684 (360)	total: 1m 6s	remaining: 5m 1s
362:	learn: 45.8575835	test: 34.5336712	best: 34.5336712 (362)	total: 1m 6s	remaining: 5m 2s
363:	learn: 45.8179966	test: 34.5293766	best: 34.5293766 (363)	total: 1m 7s	remaining: 5m 1s
364:	learn: 45.7786960	test: 34.5266580	best: 34.5266580 (364)	total: 1m 7s	remaining: 5m 2s
365:	learn: 45.7395358	test: 34.5262039	best: 34.5262039 (365)	total: 1m 7s	remaining: 5m 2s
366:	learn: 45.6999747	test: 34.5261131	best: 34.5261131 (366)	total: 1m 7s	remaining: 5m 2s
367:	learn: 45.6628969	test: 34.5223327	best: 34.5223327 (367)	total: 1m 

...

1579:	learn: 55.7717613	test: 70.0423981	best: 69.4775867 (1238)	total: 3m 20s	remaining: 53.2s
1580:	learn: 55.7582298	test: 70.0658183	best: 69.4775867 (1238)	total: 3m 20s	remaining: 53s
1581:	learn: 55.7448076	test: 70.0728822	best: 69.4775867 (1238)	total: 3m 20s	remaining: 52.9s
1582:	learn: 55.7283157	test: 70.0725463	best: 69.4775867 (1238)	total: 3m 20s	remaining: 52.8s
1583:	learn: 55.7181542	test: 70.0696347	best: 69.4775867 (1238)	total: 3m 20s	remaining: 52.6s
1584:	learn: 55.7040611	test: 70.0698274	best: 69.4775867 (1238)	total: 3m 20s	remaining: 52.5s
1585:	learn: 55.6924107	test: 70.0731315	best: 69.4775867 (1238)	total: 3m 20s	remaining: 52.4s
1586:	learn: 55.6771488	test: 70.0748768	best: 69.4775867 (1238)	total: 3m 20s	remaining: 52.2s
1587:	learn: 55.6653163	test: 70.0853770	best: 69.4775867 (1238)	total: 3m 20s	remaining: 52.1s
1588:	learn: 55.6553244	test: 70.0840719	best: 69.4775867 (1238)	total: 3m 20s	remaining: 52s
1589:	learn: 55.6416347	test: 70.0829076	bes

1666:	learn: 54.6584071	test: 70.1223635	best: 69.4775867 (1238)	total: 3m 28s	remaining: 41.6s
1667:	learn: 54.6462959	test: 70.1329230	best: 69.4775867 (1238)	total: 3m 28s	remaining: 41.4s
1668:	learn: 54.6354158	test: 70.1318825	best: 69.4775867 (1238)	total: 3m 28s	remaining: 41.3s
1669:	learn: 54.6189314	test: 70.1683705	best: 69.4775867 (1238)	total: 3m 28s	remaining: 41.2s
1670:	learn: 54.6069738	test: 70.1777198	best: 69.4775867 (1238)	total: 3m 28s	remaining: 41.1s
1671:	learn: 54.5942118	test: 70.1790233	best: 69.4775867 (1238)	total: 3m 28s	remaining: 41s
1672:	learn: 54.5792694	test: 70.1786364	best: 69.4775867 (1238)	total: 3m 28s	remaining: 40.8s
1673:	learn: 54.5637207	test: 70.1882447	best: 69.4775867 (1238)	total: 3m 29s	remaining: 40.7s
1674:	learn: 54.5525492	test: 70.1850137	best: 69.4775867 (1238)	total: 3m 29s	remaining: 40.6s
1675:	learn: 54.5425958	test: 70.1725978	best: 69.4775867 (1238)	total: 3m 29s	remaining: 40.5s
1676:	learn: 54.5325710	test: 70.1729501	b

1753:	learn: 53.5204619	test: 70.3538501	best: 69.4775867 (1238)	total: 3m 37s	remaining: 30.6s
1754:	learn: 53.5053620	test: 70.3527888	best: 69.4775867 (1238)	total: 3m 37s	remaining: 30.4s
1755:	learn: 53.4941736	test: 70.3552796	best: 69.4775867 (1238)	total: 3m 38s	remaining: 30.3s
1756:	learn: 53.4840661	test: 70.3605157	best: 69.4775867 (1238)	total: 3m 38s	remaining: 30.2s
1757:	learn: 53.4731688	test: 70.3573407	best: 69.4775867 (1238)	total: 3m 38s	remaining: 30.1s
1758:	learn: 53.4636677	test: 70.3593229	best: 69.4775867 (1238)	total: 3m 38s	remaining: 30s
1759:	learn: 53.4545511	test: 70.3759273	best: 69.4775867 (1238)	total: 3m 38s	remaining: 29.8s
1760:	learn: 53.4475658	test: 70.3775455	best: 69.4775867 (1238)	total: 3m 38s	remaining: 29.7s
1761:	learn: 53.4381523	test: 70.3745068	best: 69.4775867 (1238)	total: 3m 39s	remaining: 29.6s
1762:	learn: 53.4303629	test: 70.3738364	best: 69.4775867 (1238)	total: 3m 39s	remaining: 29.5s
1763:	learn: 53.4141516	test: 70.3953612	b

1840:	learn: 52.5030126	test: 70.5659528	best: 69.4775867 (1238)	total: 3m 48s	remaining: 19.7s
1841:	learn: 52.4911145	test: 70.5623681	best: 69.4775867 (1238)	total: 3m 48s	remaining: 19.6s
1842:	learn: 52.4802611	test: 70.5647812	best: 69.4775867 (1238)	total: 3m 48s	remaining: 19.5s
1843:	learn: 52.4706673	test: 70.5811209	best: 69.4775867 (1238)	total: 3m 48s	remaining: 19.4s
1844:	learn: 52.4621676	test: 70.5862248	best: 69.4775867 (1238)	total: 3m 48s	remaining: 19.2s
1845:	learn: 52.4506035	test: 70.6072946	best: 69.4775867 (1238)	total: 3m 48s	remaining: 19.1s
1846:	learn: 52.4355309	test: 70.5996561	best: 69.4775867 (1238)	total: 3m 49s	remaining: 19s
1847:	learn: 52.4242331	test: 70.6050892	best: 69.4775867 (1238)	total: 3m 49s	remaining: 18.8s
1848:	learn: 52.4106430	test: 70.5824322	best: 69.4775867 (1238)	total: 3m 49s	remaining: 18.7s
1849:	learn: 52.4005117	test: 70.5827030	best: 69.4775867 (1238)	total: 3m 49s	remaining: 18.6s
1850:	learn: 52.3873190	test: 70.5934234	b

1926:	learn: 51.5471119	test: 70.8203257	best: 69.4775867 (1238)	total: 3m 56s	remaining: 8.98s
1927:	learn: 51.5334433	test: 70.8191474	best: 69.4775867 (1238)	total: 3m 57s	remaining: 8.85s
1928:	learn: 51.5247347	test: 70.8292664	best: 69.4775867 (1238)	total: 3m 57s	remaining: 8.73s
1929:	learn: 51.5163789	test: 70.8310708	best: 69.4775867 (1238)	total: 3m 57s	remaining: 8.6s
1930:	learn: 51.5059899	test: 70.8468948	best: 69.4775867 (1238)	total: 3m 57s	remaining: 8.48s
1931:	learn: 51.4955978	test: 70.8423218	best: 69.4775867 (1238)	total: 3m 57s	remaining: 8.36s
1932:	learn: 51.4853398	test: 70.8459725	best: 69.4775867 (1238)	total: 3m 57s	remaining: 8.24s
1933:	learn: 51.4717675	test: 70.8526139	best: 69.4775867 (1238)	total: 3m 57s	remaining: 8.11s
1934:	learn: 51.4581960	test: 70.8512021	best: 69.4775867 (1238)	total: 3m 57s	remaining: 7.99s
1935:	learn: 51.4500153	test: 70.8620299	best: 69.4775867 (1238)	total: 3m 57s	remaining: 7.87s
1936:	learn: 51.4384289	test: 70.8642020	

# Full train

In [36]:
df_full_train = df.drop(['datetime', 'history'] + target_cols, axis=1)
y_full_train = df[target_cols]

In [37]:
dict_iterations = {5: 440, 
                   10: 565, 
                   15: 933, 
                   20: 1225, 
                   25: 1238}

In [38]:
for position in target_positions:
    model = catboost.CatBoostRegressor(
        iterations=dict_iterations[position], learning_rate=1.0, loss_function='MAE')
    model.fit(
        X=df_full_train[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]],
        y=y_full_train['target_{}'.format(position)],
        use_best_model=False,
        eval_set=(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]], y_test['target_{}'.format(position)]),
        verbose=True)
    y_predict = model.predict(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]])
    
    print('target_{}'.format(position))
    print('stupid:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_train['target_{}'.format(position)].median())))
    print('model:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_predict)))
    print()
    
    model_to_save['models'][position] = model

0:	learn: 95.3205672	test: 71.6080978	best: 71.6080978 (0)	total: 105ms	remaining: 46.2s
1:	learn: 94.8205797	test: 71.1081103	best: 71.1081103 (1)	total: 192ms	remaining: 42.1s
2:	learn: 94.3206545	test: 70.6082219	best: 70.6082219 (2)	total: 271ms	remaining: 39.5s
3:	learn: 93.8207854	test: 70.1084175	best: 70.1084175 (3)	total: 503ms	remaining: 54.8s
4:	learn: 93.3214962	test: 69.6094378	best: 69.6094378 (4)	total: 756ms	remaining: 1m 5s
5:	learn: 92.8227163	test: 69.1110889	best: 69.1110889 (5)	total: 996ms	remaining: 1m 12s
6:	learn: 92.3257525	test: 68.6151445	best: 68.6151445 (6)	total: 1.19s	remaining: 1m 13s
7:	learn: 91.8305836	test: 68.1213337	best: 68.1213337 (7)	total: 1.39s	remaining: 1m 14s
8:	learn: 91.3389204	test: 67.6317811	best: 67.6317811 (8)	total: 1.61s	remaining: 1m 16s
9:	learn: 90.8509680	test: 67.1462110	best: 67.1462110 (9)	total: 1.84s	remaining: 1m 19s
10:	learn: 90.3674312	test: 66.6662666	best: 66.6662666 (10)	total: 2.04s	remaining: 1m 19s
11:	learn: 89

90:	learn: 66.3725137	test: 45.2070652	best: 45.2070652 (90)	total: 23.6s	remaining: 1m 30s
91:	learn: 66.1866789	test: 45.0652695	best: 45.0652695 (91)	total: 23.9s	remaining: 1m 30s
92:	learn: 66.0029266	test: 44.9273586	best: 44.9273586 (92)	total: 24.1s	remaining: 1m 29s
93:	learn: 65.8205442	test: 44.7920085	best: 44.7920085 (93)	total: 24.3s	remaining: 1m 29s
94:	learn: 65.6399258	test: 44.6575095	best: 44.6575095 (94)	total: 24.5s	remaining: 1m 29s
95:	learn: 65.4610541	test: 44.5247930	best: 44.5247930 (95)	total: 24.7s	remaining: 1m 28s
96:	learn: 65.2836998	test: 44.3902613	best: 44.3902613 (96)	total: 25s	remaining: 1m 28s
97:	learn: 65.1084234	test: 44.2614573	best: 44.2614573 (97)	total: 25.2s	remaining: 1m 27s
98:	learn: 64.9344813	test: 44.1307387	best: 44.1307387 (98)	total: 25.4s	remaining: 1m 27s
99:	learn: 64.7617347	test: 44.0008098	best: 44.0008098 (99)	total: 25.6s	remaining: 1m 27s
100:	learn: 64.5903349	test: 43.8726891	best: 43.8726891 (100)	total: 25.8s	remain

178:	learn: 54.7244488	test: 37.3654519	best: 37.3654519 (178)	total: 44.1s	remaining: 1m 4s
179:	learn: 54.6311223	test: 37.3161718	best: 37.3161718 (179)	total: 44.3s	remaining: 1m 3s
180:	learn: 54.5387660	test: 37.2627268	best: 37.2627268 (180)	total: 44.5s	remaining: 1m 3s
181:	learn: 54.4471266	test: 37.2101831	best: 37.2101831 (181)	total: 44.7s	remaining: 1m 3s
182:	learn: 54.3562115	test: 37.1615626	best: 37.1615626 (182)	total: 45s	remaining: 1m 3s
183:	learn: 54.2651237	test: 37.1111861	best: 37.1111861 (183)	total: 45.2s	remaining: 1m 2s
184:	learn: 54.1751578	test: 37.0620334	best: 37.0620334 (184)	total: 45.5s	remaining: 1m 2s
185:	learn: 54.0859098	test: 37.0127374	best: 37.0127374 (185)	total: 45.7s	remaining: 1m 2s
186:	learn: 53.9966468	test: 36.9643922	best: 36.9643922 (186)	total: 46s	remaining: 1m 2s
187:	learn: 53.9081495	test: 36.9156800	best: 36.9156800 (187)	total: 46.2s	remaining: 1m 1s
188:	learn: 53.8203496	test: 36.8686535	best: 36.8686535 (188)	total: 46.5

267:	learn: 48.3371750	test: 34.4065700	best: 34.4065700 (267)	total: 1m 8s	remaining: 44.2s
268:	learn: 48.2835438	test: 34.3899217	best: 34.3899217 (268)	total: 1m 9s	remaining: 44s
269:	learn: 48.2303062	test: 34.3717560	best: 34.3717560 (269)	total: 1m 9s	remaining: 43.8s
270:	learn: 48.1773349	test: 34.3556463	best: 34.3556463 (270)	total: 1m 9s	remaining: 43.5s
271:	learn: 48.1239515	test: 34.3399825	best: 34.3399825 (271)	total: 1m 9s	remaining: 43.2s
272:	learn: 48.0710600	test: 34.3084253	best: 34.3084253 (272)	total: 1m 10s	remaining: 42.9s
273:	learn: 48.0188929	test: 34.2918020	best: 34.2918020 (273)	total: 1m 10s	remaining: 42.7s
274:	learn: 47.9662727	test: 34.2647720	best: 34.2647720 (274)	total: 1m 10s	remaining: 42.4s
275:	learn: 47.9147118	test: 34.2431946	best: 34.2431946 (275)	total: 1m 10s	remaining: 42.1s
276:	learn: 47.8632800	test: 34.2197331	best: 34.2197331 (276)	total: 1m 11s	remaining: 41.9s
277:	learn: 47.8115474	test: 34.1934317	best: 34.1934317 (277)	tota

355:	learn: 44.4975147	test: 33.0657653	best: 33.0657653 (355)	total: 1m 29s	remaining: 21.2s
356:	learn: 44.4631343	test: 33.0605861	best: 33.0605861 (356)	total: 1m 30s	remaining: 21s
357:	learn: 44.4275067	test: 33.0508924	best: 33.0508924 (357)	total: 1m 30s	remaining: 20.7s
358:	learn: 44.3934314	test: 33.0463266	best: 33.0463266 (358)	total: 1m 30s	remaining: 20.5s
359:	learn: 44.3571411	test: 33.0358598	best: 33.0358598 (359)	total: 1m 30s	remaining: 20.2s
360:	learn: 44.3229693	test: 33.0267425	best: 33.0267425 (360)	total: 1m 31s	remaining: 19.9s
361:	learn: 44.2891488	test: 33.0131904	best: 33.0131904 (361)	total: 1m 31s	remaining: 19.8s
362:	learn: 44.2540217	test: 33.0045665	best: 33.0045665 (362)	total: 1m 31s	remaining: 19.5s
363:	learn: 44.2199400	test: 32.9882074	best: 32.9882074 (363)	total: 1m 32s	remaining: 19.3s
364:	learn: 44.1860806	test: 32.9689461	best: 32.9689461 (364)	total: 1m 32s	remaining: 19s
365:	learn: 44.1532359	test: 32.9620134	best: 32.9620134 (365)	t

...

848:	learn: 76.1727438	test: 63.2010387	best: 63.2010387 (848)	total: 2m 28s	remaining: 1m 7s
849:	learn: 76.1160112	test: 63.1873637	best: 63.1873637 (849)	total: 2m 28s	remaining: 1m 7s
850:	learn: 76.0582581	test: 63.1677895	best: 63.1677895 (850)	total: 2m 28s	remaining: 1m 7s
851:	learn: 75.9992224	test: 63.1466087	best: 63.1466087 (851)	total: 2m 28s	remaining: 1m 7s
852:	learn: 75.9419651	test: 63.1295469	best: 63.1295469 (852)	total: 2m 28s	remaining: 1m 7s
853:	learn: 75.8843398	test: 63.1173904	best: 63.1173904 (853)	total: 2m 29s	remaining: 1m 7s
854:	learn: 75.8256197	test: 63.1054577	best: 63.1054577 (854)	total: 2m 29s	remaining: 1m 6s
855:	learn: 75.7678026	test: 63.0698945	best: 63.0698945 (855)	total: 2m 29s	remaining: 1m 6s
856:	learn: 75.7105724	test: 63.0456547	best: 63.0456547 (856)	total: 2m 29s	remaining: 1m 6s
857:	learn: 75.6541791	test: 62.9800009	best: 62.9800009 (857)	total: 2m 29s	remaining: 1m 6s
858:	learn: 75.5982119	test: 62.9545877	best: 62.9545877 (85

936:	learn: 71.5304942	test: 61.2050553	best: 61.2050553 (936)	total: 2m 45s	remaining: 53.1s
937:	learn: 71.4836001	test: 61.1961441	best: 61.1961441 (937)	total: 2m 45s	remaining: 53s
938:	learn: 71.4404641	test: 61.1894418	best: 61.1894418 (938)	total: 2m 45s	remaining: 52.8s
939:	learn: 71.3954899	test: 61.1747069	best: 61.1747069 (939)	total: 2m 45s	remaining: 52.6s
940:	learn: 71.3511154	test: 61.1571403	best: 61.1571403 (940)	total: 2m 46s	remaining: 52.4s
941:	learn: 71.3090440	test: 61.1510594	best: 61.1510594 (941)	total: 2m 46s	remaining: 52.3s
942:	learn: 71.2640886	test: 61.1476606	best: 61.1476606 (942)	total: 2m 46s	remaining: 52.1s
943:	learn: 71.2215226	test: 61.1447411	best: 61.1447411 (943)	total: 2m 46s	remaining: 51.9s
944:	learn: 71.1774483	test: 61.1315552	best: 61.1315552 (944)	total: 2m 46s	remaining: 51.8s
945:	learn: 71.1318359	test: 61.1241895	best: 61.1241895 (945)	total: 2m 47s	remaining: 51.6s
946:	learn: 71.0849096	test: 61.1037525	best: 61.1037525 (946)

1023:	learn: 68.1291637	test: 60.0989758	best: 60.0989758 (1023)	total: 3m	remaining: 37.6s
1024:	learn: 68.0958191	test: 60.0753886	best: 60.0753886 (1024)	total: 3m	remaining: 37.5s
1025:	learn: 68.0649828	test: 60.0636994	best: 60.0636994 (1025)	total: 3m	remaining: 37.3s
1026:	learn: 68.0341809	test: 60.0563534	best: 60.0563534 (1026)	total: 3m	remaining: 37.1s
1027:	learn: 68.0041974	test: 60.0493466	best: 60.0493466 (1027)	total: 3m	remaining: 36.9s
1028:	learn: 67.9742457	test: 60.0372755	best: 60.0372755 (1028)	total: 3m	remaining: 36.7s
1029:	learn: 67.9406119	test: 60.0134683	best: 60.0134683 (1029)	total: 3m 1s	remaining: 36.6s
1030:	learn: 67.9103240	test: 60.0121664	best: 60.0121664 (1030)	total: 3m 1s	remaining: 36.4s
1031:	learn: 67.8816888	test: 60.0027166	best: 60.0027166 (1031)	total: 3m 1s	remaining: 36.2s
1032:	learn: 67.8528434	test: 59.9995648	best: 59.9995648 (1032)	total: 3m 1s	remaining: 36s
1033:	learn: 67.8233336	test: 59.9967990	best: 59.9967990 (1033)	total

1110:	learn: 65.7258018	test: 58.8562955	best: 58.8562955 (1110)	total: 3m 13s	remaining: 22.1s
1111:	learn: 65.7034736	test: 58.8540638	best: 58.8540638 (1111)	total: 3m 13s	remaining: 21.9s
1112:	learn: 65.6801441	test: 58.8387863	best: 58.8387863 (1112)	total: 3m 13s	remaining: 21.7s
1113:	learn: 65.6577230	test: 58.8324071	best: 58.8324071 (1113)	total: 3m 13s	remaining: 21.6s
1114:	learn: 65.6317665	test: 58.8290927	best: 58.8290927 (1114)	total: 3m 13s	remaining: 21.4s
1115:	learn: 65.6029605	test: 58.8073305	best: 58.8073305 (1115)	total: 3m 13s	remaining: 21.2s
1116:	learn: 65.5811360	test: 58.7938460	best: 58.7938460 (1116)	total: 3m 14s	remaining: 21s
1117:	learn: 65.5583436	test: 58.7901869	best: 58.7901869 (1117)	total: 3m 14s	remaining: 20.8s
1118:	learn: 65.5367219	test: 58.7877416	best: 58.7877416 (1118)	total: 3m 14s	remaining: 20.7s
1119:	learn: 65.5100605	test: 58.7849665	best: 58.7849665 (1119)	total: 3m 14s	remaining: 20.5s
1120:	learn: 65.4849063	test: 58.7766035	b

1197:	learn: 63.7145570	test: 57.8241928	best: 57.8241928 (1197)	total: 3m 25s	remaining: 6.87s
1198:	learn: 63.6921301	test: 57.8210456	best: 57.8210456 (1198)	total: 3m 25s	remaining: 6.69s
1199:	learn: 63.6700430	test: 57.8201737	best: 57.8201737 (1199)	total: 3m 25s	remaining: 6.52s
1200:	learn: 63.6497948	test: 57.8145348	best: 57.8145348 (1200)	total: 3m 26s	remaining: 6.35s
1201:	learn: 63.6291908	test: 57.8086612	best: 57.8086612 (1201)	total: 3m 26s	remaining: 6.18s
1202:	learn: 63.6095441	test: 57.7878941	best: 57.7878941 (1202)	total: 3m 26s	remaining: 6.01s
1203:	learn: 63.5902508	test: 57.7823534	best: 57.7823534 (1203)	total: 3m 26s	remaining: 5.84s
1204:	learn: 63.5717422	test: 57.7769703	best: 57.7769703 (1204)	total: 3m 26s	remaining: 5.66s
1205:	learn: 63.5454501	test: 57.7654250	best: 57.7654250 (1205)	total: 3m 27s	remaining: 5.49s
1206:	learn: 63.5206591	test: 57.7543233	best: 57.7543233 (1206)	total: 3m 27s	remaining: 5.32s
1207:	learn: 63.5041029	test: 57.7489223

In [39]:
pickle.dump(model_to_save, open('set2_features_model_full_train.pkl', 'wb'))